# 2023-09-06 12 - Tables and plots for paper.ipynb


## Fetching runs

In [5]:
import wandb
from math import isnan 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import wandb
from tqdm import tqdm
# from cycler import cycler
import matplotlib as mpl
from IPython.display import Markdown, display
    
api = wandb.Api()
project = "Autoformer"
workspace = "alelab"

experiment_tags = [
    "e1_weather_10e_statinformed",
    "e2_electricity_10e_statinformed",
    "e3_weather_10e_statinformed_seeds23",
    "e4_electricity_10e_statinformed_seeds23"
    
    ]

# get all runs that both: 1.  match any experiment tag and 2. are finished
runs = api.runs(f"{workspace}/{project}",
                {"$and": [
                    {"tags": {"$in": experiment_tags}},
                    {"state": "finished"}
                ]})

all_runs = []
run_counter = 0
for run in tqdm(runs):
    run_counter += 1
    for split in ["train", "test"]:
        for metric in ["mse",]:
            pred_len = run.config["pred_len"]
            metrics = np.zeros(pred_len)
            for i in range(pred_len):
                run_dict = {**run.config}
                #run_dict["constraint_level"] = constraint_level
                run_dict[f"{metric}"] = run.summary[f"{metric}/{split}/{i}"]
                #run_dict[f"{metric}"] = run.summary.get(f"{metric}/{split}/{i}",run.summary.get(f"mse/{split}/{i}",np.nan)) #god forgive me for this line
                run_dict["step"]=i
                run_dict["epoch"]=run.summary["epoch"]
                run_dict["infeasible_rate"]=run.summary[f"infeasible_rate/{split}"]
                run_dict["split"] = split
                run_dict["run_id"] = run.id
                # Get either Constrained/ or ERM/ from the run name, then append model name.
                #print("run.name", run.name)
                #debug if ERM run
                run_dict["Algorithm"] = f"{run.name.split('/')[0]} {run.config['model']}"
                run_dict["sweep_id"] = run.sweep.id
                #print("Algorithm", run_dict["Algorithm"])

                # To better plot constrained vs ERM
                #TODO this is a hack while I consolidate the tags. 
                run_dict["type"] = "ERM" if run.config['dual_lr'] == 0 else "Constrained"

                all_runs.append(run_dict)
print(f"Fetched {run_counter} runs")
df = pd.DataFrame(all_runs)
print(f"Total records: {(df.shape)}")
print(f"Total runs: {df.run_id.nunique()}")

100%|██████████| 192/192 [00:10<00:00, 19.10it/s]


Fetched 192 runs
Total records: (129024, 62)
Total runs: 192


In [7]:
df[['run_id',"sweep_id",'Algorithm','model','constraint_level','pred_len','epoch']] \
    .drop_duplicates().sort_values(["pred_len",'Algorithm','model',"constraint_level"]) \
    .head()

,run_id,sweep_id,Algorithm,model,constraint_level,pred_len,epoch
84288,govqe5nc,lcslu9kv,Electricity-StatInformed-10e-Constrained Autof...,Autoformer,0.157,96,10
85440,o5gb1msz,3klzcp50,Electricity-StatInformed-10e-Constrained Autof...,Autoformer,0.157,96,10
101952,iij1xyjo,jk3es6p9,Electricity-StatInformed-10e-Constrained Autof...,Autoformer,0.157,96,10
83136,11eyioz9,lcslu9kv,Electricity-StatInformed-10e-Constrained Autof...,Autoformer,0.169,96,10
85056,zjzzgw4o,3klzcp50,Electricity-StatInformed-10e-Constrained Autof...,Autoformer,0.169,96,10


## Data processing
1. Impute seed for the first batch of runs
2. Create aggregate dataset of metrics per experiment `(dataset,model,pred_le,constraint_level,constrained_unconstraled)`
3. Add any other useful columns

## Plots

### ERM vs. Constrained on test split.
TO DO: Reproduce the ERM vs constrained per model for each window size. Try to unify into one plot if possible.

In [8]:
to_plot = df[df.split=='test']# just see test to unclutter
#to_plot=df
for (pred_len), series in to_plot.groupby(["pred_len"]):
    constraints = series.constraint_level.unique()
    print(constraints)
    constraints = constraints[constraints!=-1] # to iterate over all constraints explored.
    for constraint_level in constraints:
        display(Markdown(f"### pred_len: {pred_len} constraint_level: {constraint_level}"))
        plot_series = series[series["constraint_level"].isin([constraint_level,-1])] # The -1 adds the ERM baseline.

        display(plot_series.groupby(['pred_len','constraint_level','Algorithm','epoch']).size().reset_index())
        means_per_experiment = plot_series.groupby(['pred_len','constraint_level','Algorithm','model','epoch','type'])['mse'].mean().reset_index()
        #display(means_per_experiment)
        # do a pivot where model are the rows, and the columns constrained and unconstrained
        display(means_per_experiment.pivot(index=['model'], columns='type', values='mse').reset_index())

        # sns.scatterplot(x="step", y="mse", hue="Algorithm",style="split", data=plot_series)
        
        # plt.axhline(y=constraint_level, color='r', linestyle='-')
        # plt.title(f"Pred Length: {pred_len}, Constraint_level: {constraint_level}")
        # plt.show()

        # This same scatterplot but in a grid by algorithm. 
        g = sns.FacetGrid(plot_series, col="model", hue="type",col_wrap=3)
        g.map(sns.scatterplot, "step", "mse", alpha=.7) 
        g.add_legend()
        # add the hline
        for ax in g.axes:
            ax.axhline(y=constraint_level, color='r', linestyle='-')
        plt.show()


[-1.     0.556  0.17   0.169  0.553  0.157  0.516]


### pred_len: 96 constraint_level: 0.556

,pred_len,constraint_level,Algorithm,epoch,0
0,96,-1.000,StatInformed-ERM-10e Autoformer,4,96
1,96,-1.000,StatInformed-ERM-10e Autoformer,10,480
2,96,-1.000,StatInformed-ERM-10e Reformer,7,96
3,96,-1.000,StatInformed-ERM-10e Reformer,10,480
4,96,0.556,StatInformed-10e Autoformer,10,288
5,96,0.556,StatInformed-10e Reformer,10,288


ValueError: Index contains duplicate entries, cannot reshape

## ERM on multiple algorithms
As a front page chart to illustrate the problem

### Loss shaping single instance qualitative analysis

### More random explorations
Ideas: 
1. Loss over time
2. Infeasibility rate over time
3. Loss shape across epochs (see how it evolves)

## Result tables
1. Reproduce the pivot table from notebook 11, perhaps add standard deviation reports.
2. Explore and find other interesting table visualziation